In [30]:
import cv2
import numpy as np
import os
import torch
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox
from utils.plots import plot_skeleton_kpts

# 1. CLAHE untuk meningkatkan kontras
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    image_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)
    return image_clahe

In [31]:
# 2. Retinex Deillumination untuk koreksi pencahayaan
def retinex_deillumination(image, levels=5):
    def gaussian_pyramid(image, levels):
        pyramid = [image]
        for _ in range(levels - 1):
            image = cv2.pyrDown(image)
            pyramid.append(image)
        return pyramid

    def laplacian_pyramid(pyramid):
        laplacian = []
        for i in range(len(pyramid) - 1):
            upsampled = cv2.pyrUp(pyramid[i + 1], dstsize=(pyramid[i].shape[1], pyramid[i].shape[0]))
            laplacian.append(cv2.subtract(pyramid[i], upsampled))
        laplacian.append(pyramid[-1])
        return laplacian

    def reconstruct_image(laplacian):
        image = laplacian[-1]
        for i in range(len(laplacian) - 2, -1, -1):
            image = cv2.pyrUp(image, dstsize=(laplacian[i].shape[1], laplacian[i].shape[0]))
            image = cv2.add(image, laplacian[i])
        return image

    # Buat Gaussian pyramid
    pyramid = gaussian_pyramid(image, levels)

    # Buat Laplacian pyramid
    laplacian = laplacian_pyramid(pyramid)

    # Rekonstruksi gambar
    reconstructed = reconstruct_image(laplacian)

    return reconstructed

In [32]:
# 3. Preprocessing pipeline
def preprocess_image(image):
    # CLAHE untuk meningkatkan kontras
    image_clahe = apply_clahe(image)

    # Retinex untuk koreksi pencahayaan
    image_retinex = retinex_deillumination(image_clahe)

    return image_retinex

# 4. Load YOLOv7-W6 Pose model
def load_model(weights_path):
    model = attempt_load(weights_path, map_location=torch.device('cpu'))  # Gunakan 'cuda' jika GPU tersedia
    return model

In [33]:
def detect_fall(keypoints, threshold=0.5):
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6
    LEFT_HIP = 11
    RIGHT_HIP = 12
    LEFT_KNEE = 13
    RIGHT_KNEE = 14

    try:
        # Get keypoints and confidence scores
        left_shoulder = keypoints[LEFT_SHOULDER * 3: (LEFT_SHOULDER + 1) * 3]
        right_shoulder = keypoints[RIGHT_SHOULDER * 3: (RIGHT_SHOULDER + 1) * 3]
        left_hip = keypoints[LEFT_HIP * 3: (LEFT_HIP + 1) * 3]
        right_hip = keypoints[RIGHT_HIP * 3: (RIGHT_HIP + 1) * 3]
        left_knee = keypoints[LEFT_KNEE * 3: (LEFT_KNEE + 1) * 3]
        right_knee = keypoints[RIGHT_KNEE * 3: (RIGHT_KNEE + 1) * 3]

        # Check confidence scores
        if (left_shoulder[2] < threshold or right_shoulder[2] < threshold or
            left_hip[2] < threshold or right_hip[2] < threshold or
            left_knee[2] < threshold or right_knee[2] < threshold):
            return False  # Skip if any keypoint is not confident

        # Calculate average y positions
        shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
        hip_y = (left_hip[1] + right_hip[1]) / 2
        knee_y = (left_knee[1] + right_knee[1]) / 2

        # Check if hip and knee are below shoulders (fall condition)
        if hip_y > shoulder_y and knee_y > shoulder_y:
            return True
    except IndexError:
        print("Warning: Keypoints array does not have expected format")
        return False

    return False

In [34]:
def read_ground_truth(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    
    # Baris pertama: frame awal fall
    frame_start = int(lines[0].strip())
    # Baris kedua: frame akhir fall
    frame_end = int(lines[1].strip())
    
    # Baris selanjutnya: bounding box untuk setiap frame
    ground_truth = []
    for line in lines[2:]:
        line = line.strip()
        if not line:  # Skip baris kosong
            continue
        try:
            # Pastikan ada 6 nilai dalam baris
            frame_num, fall_status, x, y, w, h = map(int, line.split(','))
            ground_truth.append((frame_num, fall_status, x, y, w, h))
        except ValueError as e:
            print(f"Error parsing line: {line}. Skipping this line. Error: {e}")
    
    return frame_start, frame_end, ground_truth

In [35]:
# 7. Main pipeline untuk video
def process_video(video_path, annotation_path, model):
    # Baca ground truth
    frame_start, frame_end, ground_truth = read_ground_truth(annotation_path)

    # Baca video
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocessing frame
        processed_frame = preprocess_image(frame)

        # Deteksi pose
        result_frame = detect_pose(model, processed_frame)

        # Tampilkan ground truth bounding box (jika ada)
        if frame_count >= frame_start and frame_count <= frame_end:
            for gt in ground_truth:
                if gt[0] == frame_count:
                    x, y, w, h = gt[2], gt[3], gt[4], gt[5]
                    cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Gambar bounding box

        # Tampilkan hasil
        cv2.imshow("Result", result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

In [36]:
# 8. Main pipeline untuk dataset
if __name__ == "__main__":
    # Load model YOLOv7-W6 Pose
    weights_path = "yolov7-w6-pose.pt"  # Ganti dengan path model Anda
    model = load_model(weights_path)

    # Path ke dataset
    dataset_path = r"C:\Users\LENOVO\Documents\A Skripsi\datasets\FallDataset\Dataset"  # Ganti dengan path dataset Anda

    # Loop melalui setiap folder di dataset
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            annotation_folder = os.path.join(folder_path, "Annotation_files")
            video_folder = os.path.join(folder_path, "Videos")

            # Loop melalui setiap file di folder Annotation_files
            for annotation_file in os.listdir(annotation_folder):
                if annotation_file.endswith(".txt"):
                    annotation_path = os.path.join(annotation_folder, annotation_file)
                    video_file = annotation_file.replace(".txt", ".avi")
                    video_path = os.path.join(video_folder, video_file)

                    # Proses video
                    print(f"Processing {video_file}...")
                    process_video(video_path, annotation_path, model)

Fusing layers... 
Processing video (1).avi...
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints shape: torch.Size([])
No valid keypoints detected in this frame. Keypoints s

KeyboardInterrupt: 